In [24]:
import pandas as pd


song_data = pd.read_json("song_record.json")

In [25]:
song_data.iloc[0]

user              {'_id': {'$oid': '641b13763a63f32bb9d71460'}}
songRecord    [Z66D0F60, Z6BIWFEZ, Z6B0IC89, Z6AFUB8O, ZZF0W...
Name: 0, dtype: object

In [26]:
z = song_data["user"][0]
z.get("_id").get("$oid")

'641b13763a63f32bb9d71460'

In [27]:
c = song_data["user"].apply(pd.Series)


In [28]:
d=c["_id"].apply(pd.Series)
d

,$oid
0,641b13763a63f32bb9d71460
1,641eb85a5525ce10c474f99f


In [29]:
from collections import Counter

a = Counter(song_data["songRecord"][0])


In [30]:
a.keys()

dict_keys(['Z66D0F60', 'Z6BIWFEZ', 'Z6B0IC89', 'Z6AFUB8O', 'ZZF0WU6O', 'Z6UCWZUD', 'Z6UFF9Z6', 'ZZAAZBZE', 'ZZ9IZ7FC', 'Z6ZU6E7O', 'ZW9CFDU9', 'ZZEU80WO', 'ZW7O8W7U', 'ZWADO97U', 'Z6BU7AWO', 'Z6BWFE7F', 'Z6ABDOOI', 'Z6BU8B6O', 'ZWA7CE0B', 'ZWZD9DCB', 'ZW6CZ0OW', 'ZWZ9Z0AO', 'ZW6I7ZE7', 'Z6UBADAF', 'ZWZ9Z09E', 'ZWZ9Z0AZ', 'ZW6FODWI', 'ZW807CUD', 'ZWBUA8B8', 'ZW8WCFEA', 'Z6AFC90A', 'Z6708WAZ', 'Z69AU0IA', 'ZODCZI0O', 'ZWAD6W69', 'ZOEFE9D0', 'ZWAEIAUC', 'ZUI98BWF', 'ZZU9060B', 'ZO9D67OA', 'ZW9BBE08', 'ZW8W777E', 'Z6BIO9UF', 'Z6AFC9WD', 'Z6A9IAC7', 'ZW6D986U', 'ZWZDB788', 'ZW6W0DAW', 'ZWZ9C7EW', 'ZWZEBZCZ', 'ZW7IUW0F', 'ZWAIFBFO', 'ZZ9IZ898', 'ZW9COZ9B', 'ZW7OCDC9', 'ZU7BII9Z', 'ZW7809OU', 'Z6WDI960', 'ZW6U9UDC', 'ZW6U9UEU', 'Z69U96OW', 'ZWA80C89', 'ZWAFDUW0', 'ZUZAO6A9', 'ZW7I06CD', 'ZW7OCEOE', 'Z690CIFO', 'ZO6EIB7B', 'ZW7IUA6E', 'Z6B7AE6A', 'Z6B6CBUD', 'Z6B7AFCO', 'Z6BU6IU6', 'Z6BZWZDF', 'Z6ACA0FZ', 'Z6BOOAFC', 'Z6BO69E8', 'Z6BO6B8U', 'ZZCIOB0W'])

In [31]:
from numpy import NaN


def generate_song_record(data):
    total_df = pd.DataFrame()
    for index in range(len(data)):
        user_data = data.iloc[index]
        user_id = user_data["user"].get("_id").get("$oid")
        if (user_data["songRecord"] is not NaN):
            song_record = Counter(user_data["songRecord"])
            df = pd.DataFrame()
            df["song_id"] = song_record.keys()
            df["plays"] = song_record.values()
            df['user_id'] = user_id
            total_df = pd.concat([total_df, df])
        # user_id = user.get("_id").get("$oid")
    return total_df


In [32]:
dt = generate_song_record(song_data)
dt.set_index("user_id", inplace=True)
dt.to_csv("song_record.csv")

In [33]:
def concat_song_record(prev_song_record, current_song_record):
    concatted_song_record = pd.concat([prev_song_record, current_song_record],join='inner')
    return concatted_song_record

In [34]:
prev_song_record = pd.read_csv("song_record.csv", index_col="user_id")
current_song_record = pd.read_csv("song_record copy.csv", index_col="user_id")

concatted_song_record = concat_song_record(
    prev_song_record=prev_song_record, current_song_record=current_song_record)
concatted_song_record['plays'] = concatted_song_record.groupby(
    ['user_id', 'song_id'])['plays'].transform('sum')
# Need to drop duplicates value since concatenate two dataframe may cause duplicated value
concatted_song_record.drop_duplicates(inplace=True)
concatted_song_record.to_csv("song_record.csv")


In [35]:
song_record_data = pd.read_csv("song_record.csv")

In [36]:
from scipy.sparse import csr_matrix

wide_song_data = song_record_data.pivot(
    index="song_id", columns="user_id", values="plays").fillna(0)

wide_song_data_sparse = csr_matrix(wide_song_data.values)


In [37]:
song_record_data

,user_id,song_id,plays
0,641b13763a63f32bb9d71460,Z66D0F60,2
1,641b13763a63f32bb9d71460,Z6BIWFEZ,2
2,641b13763a63f32bb9d71460,Z6B0IC89,2
3,641b13763a63f32bb9d71460,Z6AFUB8O,2
4,641b13763a63f32bb9d71460,ZZF0WU6O,2
...,...,...,...
82,test2,ZO6EIB7B,5
83,test2,Z6UBADAF,43
84,test2,ZW807CUD,15
85,test1,ZW6CZ0OW,20


In [38]:
import numpy as np


def save_sparse_csr(filename, array):
    np.savez(filename, data=array.data, indices=array.indices,
             indptr=array.indptr, shape=array.shape)


def load_sparse_csr(filename):
    loader = np.load(filename)
    return csr_matrix((loader['data'], loader['indices'], loader['indptr']), shape=loader['shape'])

save_sparse_csr("song_data_sparse_matrix.npz",wide_song_data_sparse)


In [39]:
wide_song_data.to_csv("song_play_record.csv")

In [40]:
from sklearn.neighbors import NearestNeighbors

model_knn = NearestNeighbors(metric="cosine", algorithm='brute')

model_knn.fit(wide_song_data_sparse)


NearestNeighbors(algorithm='brute', metric='cosine')

In [41]:
import pickle
pickle.dump(model_knn,open("knn_model.sav","wb"))

In [42]:
import string
from fuzzywuzzy import fuzz

In [43]:
def print_song_recommendations(song_id, song_play_matrix, knn_model, k):
    """
    Inputs:
        song_id: id of song
        song_play_matrix: song_play_matrix (not the sparse one, the pandas dataframe)
        knn_model: our previously fitted knn_model
        k: the number of nearest neighbors
    Prints: Song recommendation for the provided song id
    Return: None
    """
    query_index = None
    ratio_tuples = []

    for i in song_play_matrix.index:
        ratio = fuzz.ratio(i, song_id)
        if ratio == 100:
            current_index_query = song_play_matrix.index.tolist().index(i)
            ratio_tuples.append((i, ratio, current_index_query))

    try:
        query_index = max(ratio_tuples, key=lambda x: x[1])[2]
        print(song_play_matrix.iloc[query_index, :])
    except:
        print("Your song didn\'t match any song id in the data. Try again")
        return None

    distances, indices = knn_model.kneighbors(
        song_play_matrix.iloc[query_index, :].values.reshape(1, -1), n_neighbors=k+1)

    for i in range(0, len(distances.flatten())):
        if i == 0:
            print("Recommendations for {0}\n".format(
                song_play_matrix.index[query_index]))
        else:
            print("{0}: {1}, with distance of {2}".format(
                i, song_play_matrix.index[indices.flatten()[i]], distances.flatten()[i]))
    return None


In [44]:
print_song_recommendations("Z6ACA0FZ", wide_song_data, model_knn, k=6)


user_id
641b13763a63f32bb9d71460    2.0
test1                       0.0
test2                       0.0
test3                       0.0
Name: Z6ACA0FZ, dtype: float64
Recommendations for Z6ACA0FZ

1: ZW9CFDU9, with distance of 0.0
2: ZW8W777E, with distance of 0.0
3: ZW7OCDC9, with distance of 0.0
4: ZW9BBE08, with distance of 0.0
5: ZW9COZ9B, with distance of 0.0
6: ZW7IUW0F, with distance of 0.0
